In [10]:
from quantum_model.system import SingleSystem, CompositeSystem
from quantum_model.dynamics import EnergyExchangeDynamics, QutritQubitExchangeDynamics

from quantum.core import DensityMatrix, GeneralQubitMatrixGen, sigmaZ
import numpy as np

## 1. System Setup

In [2]:
systemNode1 = SingleSystem(DensityMatrix(0))
systemNode1.name = 'Node1'

systemNode2 = SingleSystem(DensityMatrix(0))
systemNode2.name = 'Node2'

target = SingleSystem(GeneralQubitMatrixGen().generateThermalState(20,3))
target.name = 'Target'

system = CompositeSystem()
system.name = 'System'
system.addSubsystem(systemNode1)
system.addSubsystem(systemNode2)



In [3]:
reservoirDynamics = EnergyExchangeDynamics(systemNode1, systemNode2, 1)
system.addDynamics(reservoirDynamics)

In [4]:
model = CompositeSystem()
model.name = 'Model'

model.addSubsystem(system)
model.addSubsystem(target)

In [5]:
interfaceDynamics = QutritQubitExchangeDynamics(target, systemNode1, 1)

In [6]:
model.addDynamics(interfaceDynamics)

In [7]:
model.updateDyanamics()

In [8]:
model.evolve(2)

In [12]:
model.state.measure(sigmaZ.tensor(DensityMatrix(np.eye(6))))

0.8438077116537068

In [17]:
model.getSubsystemState(systemNode1).measure(sigmaZ)

0.8438077116537067

In [4]:
state = DensityMatrix(np.array([[1,0,0,1],[0,0,0,0],[0,0,0,0],[1,0,0,1]])*(1/2))
system.state = state


In [5]:
system.state

[[0.5 0.  0.  0.5]
 [0.  0.  0.  0. ]
 [0.  0.  0.  0. ]
 [0.5 0.  0.  0.5]]

In [6]:
system.configuration

[2, 2]

In [7]:
system.subsystemIndex(systemNode2)

[1]

In [8]:
[1,2][:0]

[]

In [26]:
system.removeSubsystem(systemNode3)

In [9]:
systemNode3.state

[[1. 0.]
 [0. 0.]]

In [27]:
system.state

[[0.     +0.j      0.     +0.j      0.     +0.j      0.     +0.j     ]
 [0.     +0.j      0.42725+0.j      0.     +0.49468j 0.     +0.j     ]
 [0.     +0.j      0.     -0.49468j 0.57275+0.j      0.     +0.j     ]
 [0.     +0.j      0.     +0.j      0.     +0.j      0.     +0.j     ]]

## 2. Reservoir Setup

In [3]:
node1 = QuantumSystem()
node1.name = 'Node1'
node1.state = DensityMatrix(np.array([[0,0],[0,1]]))

node2 = QuantumSystem()
node2.name = 'Node2'
node2.state = DensityMatrix(np.array([[0,0],[0,1]]))

reservoir = QuantumSystem()
reservoir.name = 'Reservoir'
reservoir.addSubsystem(node1)
reservoir.addSubsystem(node2)



In [4]:
exchange_func = EnergyExchangeDynamics(node1, node2, 1)
dampingNode1 = DampingDynamics(node1, 1)
dampingNode2 = DampingDynamics(node2, 1)

In [5]:
dynamics = DerivativeSystemDynamics(reservoir)
dynamics.addDynamics(exchange_func)
dynamics.addDynamics(dampingNode1)
dynamics.addDynamics(dampingNode2)

## 3. Network Setup

In [6]:
network = QuantumSystem()
network.name = 'Network'
network.addSubsystem(system)
network.addSubsystem(reservoir)

In [8]:
pumpingFunc = PumpingDynamics(systemNode1, node2, 1, 1)

AttributeError: 'NoneType' object has no attribute 'matrix'

In [ ]:
dynamics = DerivativeSystemDynamics(network)
dynamics.addDynamics(pumpingFunc)

In [ ]:
network.state

[[0 0 0 0]
 [0 1 0 0]
 [0 0 0 0]
 [0 0 0 0]]

In [ ]:
measurements = []

In [ ]:
for i in range(100):
    dynamics.evolve(1, 0.001)
    measurements.append(system.measure(sigmaZ, node2))

In [ ]:
system.state.isLegitamate()

True

In [ ]:
system.state

[[0.     +0.j      0.     +0.j      0.     +0.j      0.     +0.j     ]
 [0.     +0.j      0.23735+0.j      0.     -0.42546j 0.     +0.j     ]
 [0.     +0.j      0.     +0.42546j 0.76265+0.j      0.     +0.j     ]
 [0.     +0.j      0.     +0.j      0.     +0.j      0.     +0.j     ]]

In [ ]:
measurements

[0.6536436208700509,
 0.14550003379172027,
 -0.8438539587186819,
 0.957659480333104,
 -0.40808206185228796,
 -0.4241790072905574,
 0.9626058662971917,
 -0.8342233605439647,
 0.12796368970353983,
 0.6669380615884901,
 -0.9998433086490381,
 0.6401443395476488,
 0.16299078068619388,
 -0.8532201076599328,
 0.9524129804537633,
 -0.39185723055498584,
 -0.4401430223655654,
 0.967250588234397,
 -0.8243313311988982,
 0.11038724400861172,
 0.6800234954555436,
 -0.9993732837011468,
 0.6264444480629172,
 0.18043044908951783,
 -0.8623188721790453,
 0.9468680108218737,
 -0.3755095979802688,
 -0.45596910423125747,
 0.9715921905694338,
 -0.814180970674508,
 0.09277620486097948,
 0.6928958217226394,
 -0.9985900724540061,
 0.612548239724848,
 0.19781357371129926,
 -0.8711474008806093,
 0.9410263091349853,
 -0.3590442871933816,
 -0.4716522932673534,
 0.9756293127192314,
 -0.80377545991819,
 0.07513609125565701,
 0.7055510064254044,
 -0.9974939203524736,
 0.5984600693647424,
 0.21513470698099257,
 -0.8797